In [1]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
from keras import backend as K
import os

# to use GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# verify that a gpu is listed
K.tensorflow_backend._get_available_gpus()

# path = get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open('data/corpus.txt', encoding='utf-8') as f:
    text = f.read().lower()
    
# text = ' '.join(text.split(' ')[:1000])    

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

/home/peter/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


corpus length: 990085
total chars: 29
nb sequences: 330015
Vectorization...
Build model...
Epoch 1/60
330015/330015 [==============================] - 168s 509us/step - loss: 1.7722

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "iones
me quedo sin hablar
en esta soleda"
iones
me quedo sin hablar
en esta soleda la man en man si me en man since te tu me si la sime time to me time
tonight i can't see the one and there time
i don't way on the light
i can't see it a way i come to something i can't see that i was a little all that i come on the way
i can't way a little something i can't see the light
something i don't way on the one a seemed
i can't see the way i was a life and that's a chorus
i can't see i 
----- diversity: 0.5
----- Generating with seed: "iones
me quedo sin hablar
en esta soleda"
iones
me quedo sin hablar
en esta soleda en amo sal entro tu nu mi as un same viento amor de talo contigo
me mas mon pas quiere a prea
a a sinta pas la la 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



330015/330015 [==============================] - 176s 532us/step - loss: 1.2975

----- Generating text after Epoch: 12
----- diversity: 0.2
----- Generating with seed: "me the way
when all is ready set and don"
me the way
when all is ready set and don't be so swing
i want to see you the way i want to be
i want to be so sun
i want to come on the same to me
i can't go on and over me
i want to love you to me
i can't go on it was a sound
i can't get you i can't say that you can
the love is a sound
i can't see me all that you leave me
i want to see the way i want to love you
i want to see you there's no new love
i want to see you hear the love
the 
----- diversity: 0.5
----- Generating with seed: "me the way
when all is ready set and don"
me the way
when all is ready set and don't it go on i could go other mine
i can't go on no
can't stop the close the seey with you
the way i'll be so sunset it seems to my love
i never know i was crazy the passion
but i try to care to love
i wanna be there


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

330015/330015 [==============================] - 169s 511us/step - loss: 2.9060

----- Generating text after Epoch: 35
----- diversity: 0.2
----- Generating with seed: "mporte les mots n'ont plus d'importance
"
mporte les mots n'ont plus d'importance
h  ou t  t th t    ll tt  oue  tontot h  aa  s yo  o eu   h o   s suoo  heoe oee re     oo  r t  ue a an  tort th o  i t io th  to   t ta   t  t an   e t   ou  o nos  ot t    th on  ae  i  u t and t e i o  one a   o te  h ees  aet  ahd ct th  tt m  hoou t th  u i s o     e    a t yo  e   o i e  oth t t te t t too t ouou   i  thngo  i a ht   a  t th a d to t a ht st he  ao  t n un  to s  a t eou te
----- diversity: 0.5
----- Generating with seed: "mporte les mots n'ont plus d'importance
"
mporte les mots n'ont plus d'importance
 soo o nd de u ia et  giinge fu'arutt n ie e dl ta  
ane    e y ranbe thnis
tetn  ben   ndn t to ttat m  oe anran n  y ereoo ge inh s
fo t ahaoe a reeote sto no noa hfoy seymeest  orr t pibvo alun ttoy i  as oeee one

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



330015/330015 [==============================] - 176s 533us/step - loss: 2.9515

----- Generating text after Epoch: 47
----- diversity: 0.2
----- Generating with seed: " of a thousand invitations
you received
"
 of a thousand invitations
you received
 d e e aed n   ieeoe ie  s t ao eoe  e  e eee   ere   e h   n a i  ew e      fesa i te  oe  i   se   le  e  ye  te dn a e o we e o s  e   t a e e  t e   e  si       ho e  le  o dr eo e  i t  e a  e w l  e h ee  h e   he  e al e w ah y   osmu e ta bee   tee  e ee  ae e d d h a eee   e e n fi sd o  o  de ae  d d t e   eu ee bde e   a t e  ee  e  e o    f    s io i s uf end    e y a ee i  ce ae  e  t
----- diversity: 0.5
----- Generating with seed: " of a thousand invitations
you received
"
 of a thousand invitations
you received
 l de iaaae se md eiy
  ne to   e
e teleai wee le 
hne sn  elly t mee n see mi sysnt  oe nsetl' tsn e orl  re 
e  sa le
an ed
 y ime
wo  e  i r we ay sd
reole  s 
ibe  lee s to seisen a c  lae e ce  e rs
 ce tee nase

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



330015/330015 [==============================] - 172s 522us/step - loss: 2.9599

----- Generating text after Epoch: 48
----- diversity: 0.2
----- Generating with seed: " my side
i celebrate my life
don't try t"
 my side
i celebrate my life
don't try t   c   n ir   co t   t e h ar  o  t  u oe     s     n        ri    at a  t  ghon  e e  t  u t    i   tht ue  t   e    t t re   a   i o   tr t r e t'    t  t t   i   nut
  g ne   b  o  ot e  th t  t  et l ic t th t  he we a   m tei t    e    i  it  t      th   to   w  e    i ma   i wt   ii t  au e t i t tat   m  t  g toue  iou t ot     ot n  m eu  t  si inn  t  t  i   t o  t  a te t  to        t  c
----- diversity: 0.5
----- Generating with seed: " my side
i celebrate my life
don't try t"
 my side
i celebrate my life
don't try t etrmntn mas yog eyan  me b m  i ti  m h l  i   dwnn  t ar   h h i otet  a ti vg ntio e
  ne o  e ee oeea eit
  tn   o t e t n   l
 e et  t
 te sr
re e  el uthrewtr c ceo y  rd  m tsisn otauwo odn  ey e r   htoe ee  